In [1]:
from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent
from langchain.tools import Tool, BaseTool
from pydantic.v1 import BaseModel, Field
# from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
import math
from scipy.stats import norm

def rw_corp(Input_String: str) -> str:

    """ 
    calculates the risk weight from input parameters 
    PD, LGD, MATURITY, F_LARGE_FIN and SIZE which are fed into the formula in this particular order 
    """
    
    PD_s, LGD_s, MATURITY_s, SIZE_s, F_LARGE_FIN_s = Input_String.split(",\n")

    PD = float(eval(PD_s.split(":")[1].strip()))
    LGD = float(eval(LGD_s.split(":")[1].strip()))
    MATURITY = float(eval(MATURITY_s.split(":")[1].strip()))
    SIZE = float(eval(SIZE_s.split(":")[1].strip()))
    F_LARGE_FIN = (F_LARGE_FIN_s.split(":")[1].strip())
       
    pd_final = max(0.0003, PD)
    size_final = max(5, min(SIZE, 50))    
    r0 = (0.12 * (1.0 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0))) + \
        (0.24 * (1.0 - (1 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0)))) - \
        (0.04 * (1.0 - (size_final - 5.0) / 45.0))
    if F_LARGE_FIN == 'Y':
        r = r0 * 1.25
    else:
        r = r0
    b = (0.11852 - 0.05478 * math.log(pd_final)) ** 2
    ma = ((1 - 1.5 * b) ** -1) * (1 + (MATURITY - 2.5) * b)    
    rw = ((LGD * norm.cdf((1 - r) ** -0.5 * norm.ppf(pd_final) + (r / (1 - r)) ** 0.5 * norm.ppf(0.999)) - pd_final * LGD) * ma) * 12.5 * 1.06
    return rw  


class RWInput(BaseModel):
    Input_String: str = Field(description='This is a string that contains values for the input parameters \
                              PD, LGD, MATURITY, SIZE and F_LARGE_FIN which are fed into the formula in this particular order ')
    
RWTool = Tool.from_function(
    func=rw_corp,
    name="RWTool",
    description="""
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution,
        
        """,
    args_schema=RWInput
)

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-4")
# llm = ChatOpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///./portfolio_data.db",
                         include_tables=['corporate_portfolio']
)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

SQLTool = Tool(
        name="db_query_tool",
        func=db_chain.run,
        description="useful for quering a database"
)

In [11]:
tools = [SQLTool, RWTool]

# memory = ConversationBufferMemory(memory_key="chat_history")

# agent = initialize_agent(
#     agent='conversational-react-description', 
#     tools=tools, 
#     llm=llm,
#     verbose=True,
#     max_iterations=3,
#     memory=memory,
#     handle_parsing_errors=True
# )


agent = initialize_agent(
    tools, 
    llm, 
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    agent="zero-shot-react-description", 
    verbose=True, 
    handle_parsing_errors=True
)

In [12]:
# output = agent.run("What are total RWA and EAD in table corporate_portfolio?")
output = agent.run("What are total RWA and total EAD in the corporate_portfolio table for reporting date 31-12-2022?")



> Entering new AgentExecutor chain...
I need to query the corporate_portfolio table in the database to get the total RWA (Risk Weighted Assets) and total EAD (Exposure At Default) for the reporting date 31-12-2022.
Action: db_query_tool
Action Input: SELECT SUM(RWA), SUM(EAD) FROM corporate_portfolio WHERE reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT SUM(RWA), SUM(EAD) FROM corporate_portfolio WHERE reporting_date = '31-12-2022'
SQLQuery:SELECT SUM("RWA"), SUM("EAD") FROM corporate_portfolio WHERE "REPORTING_DATE" = '2022-12-31'
SQLResult: [(85075812.12498187, 156657332.34921697)]
Answer:The sum of RWA is 85075812.12 and the sum of EAD is 156657332.35 on 31-12-2022.
> Finished chain.

Observation: The sum of RWA is 85075812.12 and the sum of EAD is 156657332.35 on 31-12-2022.
Thought:I now know the final answer
Final Answer: The total RWA in the corporate_portfolio table for reporting date 31-12-2022 is 85075812.12 and the total EAD is 156657332.35.


In [13]:
output = agent.run("Read the information in the corporate_portfolio table for loan id dninx621xv9731 and for reporting date 31-12-2022 and then calculate the RW if PD doubles")



> Entering new AgentExecutor chain...
First, I need to fetch the required information from the corporate_portfolio table for the specified loan id and reporting date.
Action: db_query_tool
Action Input: SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'
SQLQuery:SELECT "REPORTING_DATE", "PARTNER_ID", "LOAN_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1439964799kgeq68' AND "REPORTING_DATE" = '2022-12-31'
SQLResult: [('2022-12-31', 'dninx621xv9731', '1439964799kgeq68', 'CREDIT_CARD', 'Performing', 5141.569897, 4578.250143, 9719.82004, 7061.256068, 0.0, 0.0156611, 0.7026, 2.345659392, 1.41369863, 'N', 1.196209223, 8446.739634)]
Answer:The details

In [14]:
output = agent.run("Read the information in the corporate_portfolio table for loan id dninx621xv9731 and for reporting date 31-12-2022 and then calculate the RW if LGD doubles")



> Entering new AgentExecutor chain...
First, I need to fetch the required information from the corporate_portfolio table for the specified loan id and reporting date.
Action: db_query_tool
Action Input: SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'
SQLQuery:SELECT "REPORTING_DATE", "PARTNER_ID", "LOAN_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1439964799kgeq68' AND "REPORTING_DATE" = '2022-12-31'
SQLResult: [('2022-12-31', 'dninx621xv9731', '1439964799kgeq68', 'CREDIT_CARD', 'Performing', 5141.569897, 4578.250143, 9719.82004, 7061.256068, 0.0, 0.0156611, 0.7026, 2.345659392, 1.41369863, 'N', 1.196209223, 8446.739634)]
Answer:The details

ValueError: not enough values to unpack (expected 5, got 4)

In [15]:
output = agent.run("Read the information in the corporate_portfolio table for loan id dninx621xv9731 and for reporting date 31-12-2022 and then calculate the RW if MATURITY becomes 6.23")



> Entering new AgentExecutor chain...
First, I need to fetch the required information from the corporate_portfolio table for the specified loan id and reporting date.
Action: db_query_tool
Action Input: SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT * FROM corporate_portfolio WHERE loan_id = 'dninx621xv9731' AND reporting_date = '31-12-2022'
SQLQuery:SELECT "REPORTING_DATE", "PARTNER_ID", "LOAN_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1439964799kgeq68' AND "REPORTING_DATE" = '2022-12-31'
SQLResult: [('2022-12-31', 'dninx621xv9731', '1439964799kgeq68', 'CREDIT_CARD', 'Performing', 5141.569897, 4578.250143, 9719.82004, 7061.256068, 0.0, 0.0156611, 0.7026, 2.345659392, 1.41369863, 'N', 1.196209223, 8446.739634)]
Answer:The details

In [18]:
output = agent.run("Read the information in the corporate_portfolio table for loan id 1851285872eypk20 and for reporting date 31-12-2022 and then calculate the RW if LGD doubles")



> Entering new AgentExecutor chain...
First, I need to fetch the required information from the corporate_portfolio table for the specified loan id and reporting date.
Action: db_query_tool
Action Input: SELECT * FROM corporate_portfolio WHERE loan_id = '1851285872eypk20' AND reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT * FROM corporate_portfolio WHERE loan_id = '1851285872eypk20' AND reporting_date = '31-12-2022'
SQLQuery:SELECT "REPORTING_DATE", "PARTNER_ID", "LOAN_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1851285872eypk20' AND "REPORTING_DATE" = '2022-12-31' LIMIT 5
SQLResult: [('2022-12-31', 'vzkah667hw5846', '1851285872eypk20', 'CREDIT_CARD', 'Performing', 0.0, 3830.21812, 3830.21812, 1565.410146, 0.0, 0.0022528, 0.549, 44.9952041, 1.0, 'N', 0.409196683, 640.5606391)]
Answer:The details for t

ValueError: not enough values to unpack (expected 5, got 4)

In [17]:
output = agent.run("Read the information in the corporate_portfolio table for loan id 1851285872eypk20 and for reporting date 31-12-2022 and then calculate the RW if LGD doubles and PD becomes 0.2")



> Entering new AgentExecutor chain...
First, I need to fetch the required information from the corporate_portfolio table for the specified loan id and reporting date.
Action: db_query_tool
Action Input: SELECT * FROM corporate_portfolio WHERE loan_id = '1851285872eypk20' AND reporting_date = '31-12-2022'

> Entering new SQLDatabaseChain chain...
SELECT * FROM corporate_portfolio WHERE loan_id = '1851285872eypk20' AND reporting_date = '31-12-2022'
SQLQuery:SELECT "REPORTING_DATE", "PARTNER_ID", "LOAN_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1851285872eypk20' AND "REPORTING_DATE" = '2022-12-31' LIMIT 5
SQLResult: [('2022-12-31', 'vzkah667hw5846', '1851285872eypk20', 'CREDIT_CARD', 'Performing', 0.0, 3830.21812, 3830.21812, 1565.410146, 0.0, 0.0022528, 0.549, 44.9952041, 1.0, 'N', 0.409196683, 640.5606391)]
Answer:The details for t

In [ ]:
output = agent.run("Read the information for ACCOUNT_NUMBER = '1851285872eypk20' in rwa_data table")

In [ ]:
output = agent.run("What will be the RW if MATURITY is 5?")

In [ ]:
output = agent.run("What is the RW for LOAN_ID dninx621xv9731 for reporting date 31-12-2022?")

In [ ]:
output = agent.run("Read the info for LOAN_ID dninx621xv9731 for reporting date 31-12-2022")

In [ ]:
output = agent.run("What will be the RW if PD doubles?")

In [ ]:
output = agent.run("What is the difference between the sum of RWA for reporting date 2022-12-31 and the sum of RWA for reporting date 2023-03-31?")